In [ ]:
import torch.nn as nn
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

### Classification classes number setting

In [ ]:
#datast_label = 'point_history_classifier_label.csv'
datast_label = 'landmark_points_labels.csv'
count = 0
with open(datast_label, 'r') as file:
  for line in file:
    count += 1
# Can change this later when updated labels
number_classes = count
number_classes

5

### Choosing the device

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

### Each path to csv


In [ ]:
# data = 'point_history.csv'
data_of_mediapipe = 'landmark_points.csv'

### Reading the csv files and loading the learning data


https://www.w3schools.com/python/python_file_open.asp

In [ ]:
# Features for my model
X_data = []
# Labels for my model
y_data = []
def is_valid_float(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

with open(data_of_mediapipe, 'r') as file:
    for i, line in enumerate(file, start=1):
        values = line.strip().split(',')
        if not all(is_valid_float(v) for v in values[1:]):
            print(f"Invalid features on line {i}: {line.strip()}")

with open(data_of_mediapipe, 'r') as file:
  for line in file:
    values = line.strip().split(',')
    label = int(values[0])  # First element as label
    features = [float(v) for v in values[1:]]  # Remaining as features
    X_data.append(features)
    y_data.append(label)
    print(f"Label: {label}, Features: {features}")

Output hidden; open in https://colab.research.google.com to view.

### Convert lists to PyTorch tensors

https://discuss.pytorch.org/t/best-way-to-convert-a-list-to-a-tensor/59949

In [ ]:
# Finding the maximum number of features
max_features = len(X_data[0])
for feature in X_data:
  max_features = max(max_features, len(feature))

print("input size: ", max_features)


# Pad shorter feature lists with zeros for consistent tensor shape
for features in X_data:
    features += [0.0] * (max_features - len(features))




X_tensor = torch.tensor(X_data, dtype=torch.float32)
y_tensor = torch.tensor(y_data, dtype=torch.long)
y_tensor

input size:  42


tensor([0, 0, 0,  ..., 4, 4, 4])

### Feed forward model


In [ ]:
class Feedforward(nn.Module):

    def __init__(self, input_size, num_classes):

      super().__init__()

      # nn.Sequential is a container that allows to build neural networks
      # in a sequential, layer-by-layer format.
      self.ff = nn.Sequential (
          # I'm guessing the number of inputs of labels?
          #
          # Have 4 labels for now Default, Moving Cursor, Scroll Down, Scroll Up

          # CHANGED nn.Linear(input_size, 20) TO nn.Linear(input_size, 100) FOR NO MISMATCH
          nn.Linear(input_size, 100), # Input layer with the max length of the feature list, intermediate with 20 nodes
          nn.ReLU(), # Activation function
          nn.Linear(100, 100), # Intermediate layer with 100 nodes
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, 100),
          nn.ReLU(),
          nn.Linear(100, num_classes), # Output layer with `num_classes` nodes
          nn.Softmax(dim=1)  # Softmax for probabilities
      )

    def forward(self, x):
        return self.ff(x)

### Insatantiate the model😈

In [ ]:
print(f"Number of classes: {number_classes}")
print(f"Max features: {max_features}")

instantiated_model = Feedforward(input_size=max_features, num_classes=number_classes).to(device)

Number of classes: 5
Max features: 42


In [ ]:
class GestureDataset(Dataset):
    def __init__(self, data_file, transform=None):
        # train_loadertrain_loader
        # data_file (str): Path to the CSV file containing the data.
        # transform (callable, optional): A function/transform to apply to the features.
        self.data = []
        self.labels = []
        self.transform = transform

        with open(data_file, 'r') as file:
            for line in file:
                # Split the line into components
                values = line.strip().split(',')
                label = int(values[0])  # The first value is the label
                features = list(map(float, values[1:]))  # Remaining values are features
                self.labels.append(label)
                self.data.append(features)

        # Convert to tensors
        self.data = torch.tensor(self.data, dtype=torch.float32)
        self.labels = torch.tensor(self.labels, dtype=torch.long)


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # Return a single data sample and its label
        return self.data[idx], self.labels[idx]

# Example usage
data_file = data_of_mediapipe

# Optional normalization transform
def normalize(tensor):
    return (tensor - tensor.mean(dim=0)) / tensor.std(dim=0)

# Create dataset instance
gesture_dataset = GestureDataset(data_file)

# Wrap in DataLoader for batching
data_loader = DataLoader(gesture_dataset, batch_size=32, shuffle=True)

# Test loading data
for features, labels in data_loader:
    print("Features batch shape:", features.shape)
    print("Labels batch shape:", labels.shape)
    break

Features batch shape: torch.Size([32, 42])
Labels batch shape: torch.Size([32])


In [ ]:
#Use the GPU if possible
instantiated_model.to(device)
# Data
training_data = gesture_dataset

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func
    def __len__(self):
        return len(self.dl)
    def __iter__(self):
        batches = iter(self.dl)
        for batch in batches:
            yield (self.func(*batch))

#device defined in previous module
def toGPU(x, y):
    return x.to(device), y.to(device)

train_loader = WrappedDataLoader(DataLoader(
    training_data, batch_size=100, shuffle=True, num_workers=1
), toGPU)


In [ ]:
from torch.utils.tensorboard import SummaryWriter
import os


writer = SummaryWriter()

# Define the path to save checkpoints
save_path_way = 'model2'

# Ensure the directory exists
os.makedirs(save_path_way, exist_ok=True)

def train(model, loader, loss_function, optimizer, num_epochs=50, save_path=save_path_way):
    model.train()

    total_step = len(loader)
    for epoch in range(num_epochs):
        total_loss = 0.0
        # stochastic gradient descent
        for i, (coordinate, label) in enumerate(loader):
            output = model(coordinate)

            loss = loss_function(output, label)

            optimizer.zero_grad() #Reset Gradients
            loss.backward() #Calculate backpropagation gradients

            optimizer.step() #Perform parameter update step

            total_loss += loss.item()

            if (i+1) % 100 == 0:
                print('Epoch {}/{}, Step {}/{}, Loss: {:.4f}'
                      .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

        checkpoint = {
            'model': model.state_dict(),
            'loss': loss
        }
        torch.save(checkpoint, save_path + '/epoch_' + str(epoch+1) + '.pt')

In [53]:
from torch import optim
loss_func = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(instantiated_model.parameters(), lr=1e-3)

train(instantiated_model, train_loader, loss_func, optimizer)